# Аерофлот
Задача складається в написані структури літаків. Літаки бувають різних типів:
- розвідники
- винищувачі
- бомбордувальники

## Особливості
В залежності від того який літак імплементовано, такі функції він має мати. Бомбордувальники можуть зкидати бомби на бази внизу. Винищувачі можуть знищувати інші літаки. Розвідники допомагають знайти літаки та бази і спрощують їхне винищення.

## Завдання
Реалізуйте весь представлений вище функціонал в певному виді використовуючи функції, класи та ООП.

In [7]:
import random
import time

class Health:
    __health = 100
    __max_health = 100

    def get_health(self):
        return self.__health

    def set_health(self, health):
        if health > self.__max_health:
            self.__health = self.__max_health
        else:
            self.__health = health

    def __init__(self, max_health=100):
        self.__max_health = max_health
        if self.__health > self.__max_health:
            self.__health = self.__max_health

    def damage(self, damage):
        self.__health -= damage
        if self.__health < 0:
            self.__health = 0

class Position:
    __position = [0, 0]
    __move_speed = 1

    def get_position(self):
        return self.__position

    def move(self, x, y):
        if x > 1:
            x = 1
        elif x < -1:
            x = -1
        if y > 1:
            y = 1
        elif y < -1:
            y = -1

        self.__position[0] += x * self.__move_speed
        self.__position[1] += y * self.__move_speed

    def __init__(self, x=0, y=0, move_speed=1):
        self.__position = [x, y]
        self.__move_speed = move_speed

class Name:
    __name = "default"
    __id = -1

    def get_name(self):
        return self.__name
    
    def get_id(self):
        return self.__id

    def __init__(self, id=0, name="plane"):
        if name == "plane":
            if id <= 35:
                name = f"Fighter jet [{id}]"
            elif 35 < id <= 75:
                name = f"Bomber [{id}]"
            elif id > 75:
                name = f"Scout [{id}]"
            else:
                name = f"Unknown plane [{id}]"
        else:
            name = f"Base [{id}]"
        self.__name = name
        self.__id = id

class Attack:
    __damage = 10

    def get_damage(self):
        return self.__damage

    def attack(self, target: Health):
        target.damage(self.__damage)

    def __init__(self, damage=10):
        self.__damage = damage


class Spot:
    __spotted = False

    def get_spot_status(self):
        return self.__spotted
    
    def set_spot_status(self, spot=True):
        self.__spotted = spot
    
    def __init__(self, spot=False):
        self.__spotted = spot

def generate_plane():
    random_value = random.randint(0, 100)
    if random_value <= 35:

        class Plane(Health, Position, Name, Attack):
            def move(self, x, y):
                if self.get_health() > 0:
                    return super().move(x, y)

            def __init__(self, max_health=100, x=0, y=0, move_speed=1, damage=10):
                Health.__init__(self, max_health)
                Position.__init__(self, x, y, move_speed)
                Name.__init__(self, id=random_value)
                Attack.__init__(self, damage)

        return Plane
    elif 40 < random_value <= 75:

        class Plane(Health, Name, Position, Attack):
            def move(self, x, y):
                if self.get_health() > 0:
                    return super().move(x, y)

            def __init__(self, max_health=100, x=0, y=0, move_speed=1, damage=10):
                Health.__init__(self, max_health)
                Position.__init__(self, x, y, move_speed)
                Name.__init__(self, id=random_value)
                Attack.__init__(self, damage)

        return Plane
    elif random_value > 75:

        class Plane(Health, Position, Name):
            def move(self, x, y):
                if self.get_health() > 0:
                    return super().move(x, y)

            def __init__(self, max_health=100, x=0, y=0, move_speed=2):
                Health.__init__(self, max_health)
                Position.__init__(self, x, y, move_speed)
                Name.__init__(self, id=random_value)

        return Plane


def generate_base():
    random_value = random.randint(101, 111)

    class Base(Health, Position, Name, Spot):
        def __init__(
            self,
            max_health=250,
            x=random.randint(-10, 10),
            y=random.randint(-10, 10),
            move_speed=0,
            spot=False
        ):
            Health.__init__(self, max_health)
            Position.__init__(self, x, y, move_speed)
            Name.__init__(self, id=random_value, name="base")
            Spot.__init__(self, spot)

    return Base

planes = []
bases = []
players_num = 10
bases_num = 3
for i in range(players_num):
    def create_plane():
        plane = generate_plane()()
        plane_id = plane.get_id()
        for temp_plane in planes:
            if(temp_plane.get_id() == plane_id):
                create_plane()
        plane.set_health(100)
        planes.append(plane)
        time.sleep(0.1)

    create_plane()

for i in range(bases_num):
    def create_base():
        base = generate_base()()
        base_id = base.get_id()
        for temp_base in bases:
            if(temp_base.get_id() == base_id):
                create_base()
        base.set_health(250)
        bases.append(base)
        time.sleep(0.1)

    create_base()

while True:
    for plane in planes:
        plane.move(random.randint(-1, 1), random.randint(-1, 1))

    crashed_planes = []
    for plane in planes:
        if plane.get_health() == 0:
            crashed_planes.append(plane)

    destroyed_bases = []
    for base in bases:
        if base.get_health() == 0:
            destroyed_bases.append(base)

    if len(destroyed_bases) == bases_num:
        print('Every base was destroyed')
        break
            

    if len(crashed_planes) >= (players_num - 1):
        for plane in planes:
            if plane.get_health() > 0:
                print(
                    f"{plane.get_name()} : {plane.get_health()} ♥️ : {plane.get_position()} - Survived"
                )
                break
        break

    for plane in planes:
            
        plane_id = plane.get_id()

        if plane.get_health() == 0:
            pass
        else:
            if plane_id <= 35:
                def create_attack_target():
                    attack_target = random.choice(planes)
                    if attack_target.get_health() == 0:
                        create_attack_target()
                    return attack_target

                target = create_attack_target()

                if target.get_id() == plane_id:
                    pass
                else:
                    plane.attack(target)
                
            elif 35 < plane_id <= 75:
                def create_attack_target():
                    attack_target = random.choice(bases)
                    if(len(destroyed_bases) >= bases_num):
                        pass
                    else:
                        if attack_target.get_health() == 0:
                            create_attack_target()
                        return attack_target

                target = create_attack_target()

                if target.get_spot_status() == False:
                    pass
                else:
                    plane.attack(target)
            else:
                def spot_the_base():
                    spotted_base = random.choice(bases)
                    status = spotted_base.get_spot_status()
                    if status == True:
                        spot_the_base()
                    spotted_base.set_spot_status()

                bases_revealed = False

                for base in bases:
                    status = base.get_spot_status()
                    if status == False:
                        bases_revealed = False
                    else:
                        bases_revealed = True

                if bases_revealed == False:

                    random_num = random.randint(0, 100)

                    if random_num < 20:
                        spot_the_base()
                

    for plane in planes:
        if plane.get_health() == 0:
            print(f"{plane.get_name()} : plane crashed : {plane.get_position()}")
        else:
            print(
                f"{plane.get_name()} : {plane.get_health()} ♥️ : {plane.get_position()}"
            )

    base_position = 'No data'
    for base in bases:
        if base.get_health() == 0:
            print(f"{base.get_name()} : base destroyed : {base.get_position()}")
        else:
            status = base.get_spot_status()
            if status == True:
                base_position = base.get_position()
            else:
                base_position = 'No data'
            print(
                f"{base.get_name()} : {base.get_health()} ♥️ : {base_position}"
            )
    print("--------------------------------")

    if input() == "q":
        break

Bomber [64] : 100 ♥️ : [-1, 0]
Bomber [56] : 100 ♥️ : [-1, 1]
Fighter jet [18] : 80 ♥️ : [1, -1]
Fighter jet [21] : 100 ♥️ : [-1, 0]
Bomber [50] : 100 ♥️ : [-1, -1]
Scout [93] : 100 ♥️ : [-2, 2]
Bomber [46] : 100 ♥️ : [1, 0]
Bomber [67] : 90 ♥️ : [-1, 1]
Scout [77] : 100 ♥️ : [-2, 0]
Fighter jet [30] : 100 ♥️ : [-1, 0]
Base [104] : 250 ♥️ : No data
Base [110] : 250 ♥️ : No data
Base [111] : 250 ♥️ : No data
Base [104] : 250 ♥️ : No data
--------------------------------
Bomber [64] : 100 ♥️ : [0, -1]
Bomber [56] : 100 ♥️ : [-1, 0]
Fighter jet [18] : 80 ♥️ : [1, -2]
Fighter jet [21] : 100 ♥️ : [-1, -1]
Bomber [50] : 100 ♥️ : [-2, -2]
Scout [93] : 100 ♥️ : [0, 2]
Bomber [46] : 100 ♥️ : [2, 1]
Bomber [67] : 80 ♥️ : [-2, 2]
Scout [77] : 100 ♥️ : [0, 0]
Fighter jet [30] : 100 ♥️ : [-2, -1]
Base [104] : 250 ♥️ : No data
Base [110] : 250 ♥️ : No data
Base [111] : 250 ♥️ : No data
Base [104] : 250 ♥️ : No data
--------------------------------
Bomber [64] : 100 ♥️ : [1, -1]
Bomber [56] : 100 ♥️ 